In [1]:
import pandas as pd
import requests
import glob
import os

import ipywidgets as widgets
from IPython.display import display

In [2]:
def build_league_table(df, live_scores):
    
    df_analysis = pd.merge(
        df,
        live_scores,
        how='outer',
        on=['Home']
    )

    df_analysis['person_direction'] = df_analysis.apply(person_direction, axis=1)
    df_analysis['live_direction'] = df_analysis.apply(live_direction, axis=1)
    df_analysis['points'] = df_analysis.apply(calculate_points, axis=1)
    
    return df_analysis.groupby('Name')['points'].sum().to_frame().sort_values('points', ascending=False).reset_index()

In [3]:
def person_direction(row):
    
    if row['Home score'] > row['Away score']:
        return 'H'
    elif row['Home score'] < row['Away score']:
        return 'A'
    else:
        return 'D'

In [4]:
def live_direction(row):
    
    if row['home_live'] > row['away_live']:
        return 'H'
    elif row['home_live'] < row['away_live']:
        return 'A'
    else:
        return 'D'

In [5]:
def calculate_points(row):
    
    if row['person_direction'] == row['live_direction']:
        if (row['Home score'] == row['home_live']) & (row['Away score'] == row['away_live']):
            return 3
        else:
            return 1
    else:
        return 0

In [6]:
files = glob.glob("./predictions/*.xlsx")

predictions_everyone = []
for f in files:
    predictions = pd.read_excel(f,engine='openpyxl',)
    predictions['Name'] = os.path.basename(f).split('.')[0]
    predictions_everyone += [predictions]
    
df = pd.concat(predictions_everyone)
df = df.fillna(value=0, inplace=False)
df['Joker'] = df['Joker'].astype(int)

In [7]:
live_scores = pd.read_excel("./live/live_scores.xlsx",engine='openpyxl')
live_scores.columns = ['Home','home_live','v','away_live','Away']

# Boxing Day Football Predictions 2020

## How they stand!

In [8]:
button = widgets.Button(description="Refresh Table")
output = widgets.Output()

display(button, output)

def on_league_button_clicked(b):
    with output:
        display(build_league_table(df, live_scores))

button.on_click(on_league_button_clicked)

Button(description='Refresh Table', style=ButtonStyle())

Output()

## Look up someone's predictions

In [9]:
options = df['Name'].unique()
person_dropdown = widgets.Dropdown(
    options=options,
    value=options[0],
    description='Person',
    disabled=False,
)
display(person_dropdown)

Dropdown(description='Person', options=('Dave', 'Dano', 'Ben'), value='Dave')

In [10]:
person_button = widgets.Button(description="Refresh")
output = widgets.Output()

display(person_button, output)

def on_person_button_clicked(b):
    with output:
        display(df.loc[df['Name']==person_dropdown.value].drop('Name', axis=1))

person_button.on_click(on_person_button_clicked)

Button(description='Refresh', style=ButtonStyle())

Output()

## Look up the predictions for a fixture

In [11]:
options = df['Home'].unique() 
fixture_dropdown = widgets.Dropdown(
    options=options,
    value=options[0],
    description='Fixture:',
    disabled=False,
)

display(fixture_dropdown)

Dropdown(description='Fixture:', options=('Leicester City', 'Burnley', 'Chelsea'), value='Leicester City')

In [12]:
fixture_button = widgets.Button(description="Refresh")
output = widgets.Output()

display(fixture_button, output)

def on_fixture_button_clicked(b):
    with output:
        display(df.loc[df['Home']==fixture_dropdown.value].drop('Name', axis=1))

fixture_button.on_click(on_fixture_button_clicked)

Button(description='Refresh', style=ButtonStyle())

Output()